In [ ]:
# INFO 300
# Muatasim Qazi
# Scrap UW Courses Descriptions
# 12/10/2018

In [375]:
# Import libraries
import re
import json
import requests
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

In [376]:
# nltk.download()

In [377]:
pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')

In [378]:
def parse(url, courses_list, dept_name):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    get_info_courses(soup, courses_list, dept_name)


In [379]:
def get_info_courses(soup, course_list, dept_name):
    
    course_titles = soup.find_all('b')
    dept_course_list = []
    for title in course_titles:
        
        course_title = title.text[8:title.text.index('(')].strip()
        course_number = title.text[:8].strip()
        course_credits = re.search(r'\((.*?)\)', title.text).group(1)
        
        requirements = title.text[title.text.index(')') + 2:].strip()
        
        course_description = title.next_sibling

        skills = []

        if course_description.name == 'br':
            course_description.decompose()
            course_description = title.next_sibling

        if course_description.name == 'i':
            course_description.decompose()
            title.next_sibling.decompose()
            course_description = title.next_sibling

            
        if course_description.name == 'a':
            course_description.decompose()
            course_description = title.next_sibling

            continue #cse improper tag <a> for one course
            
            
        skills_from_desc = course_description.replace(".", "").split(",")

        gen = (skill for skill in skills_from_desc if len(skill.split(" ")) < 5)


        for skill in gen:
            text = pattern.sub('', skill)
            skills.append(text.strip())


        course = {
            "title": course_title,
            "number": course_number,
            "credits": course_credits,
            "requirements": requirements,
            "description": course_description,
            "skills": skills
        }
    
        dept_course_list.append(course)
    course_list.append({dept_name.upper(): dept_course_list})
    

In [380]:
uw_cs_depts = ['info', 'cse', 'hcde']
uw_all_cs_courses = []

In [381]:
for dept in uw_cs_depts:
    uw_courses_url = f'https://www.washington.edu/students/crscat/{dept}.html'
    parse(uw_courses_url, uw_all_cs_courses, dept)


In [382]:
uw_all_cs_courses

[{'INFO': [{'title': 'Social Networking Technologies',
    'number': 'INFO 101',
    'credits': '5',
    'requirements': 'I&S/NW',
    'description': "Explores today's most popular social networks, gaming applications, and messaging applications. Examines technologies, social implications, and information structure. Focuses on logic, databases, networked delivery, identity, access, privacy, ecommerce, organization, and retrieval.",
    'skills': ['gaming applications',
     'social implications',
     'databases',
     'networked delivery',
     'identity',
     'access',
     'privacy',
     'ecommerce',
     'organization',
     'retrieval']},
   {'title': 'Gender and Information Technology',
    'number': 'INFO 102',
    'credits': '5',
    'requirements': 'I&S, DIV',
    'description': 'Explores the social construction of gender in relation to the history and contemporary development of information technologies. Considers the importance of diversity and difference in the design and

In [383]:
data = uw_all_cs_courses

In [384]:
# write the clean data to a new json file
with open('data_courses_skills.json', 'w') as outfile:
    json.dump(data, outfile)